In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [41]:
import re
import nltk
import string
import gensim
def tokenize(text):
    stem = nltk.stem.SnowballStemmer('english')
    text = text.lower()
    for token in nltk.word_tokenize(text): #word_tokenize() to split a sentence into words.
        if token in string.punctuation: continue   #string.punctuation='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        #yield stem.stem(token)              # here yield command i sused as length of array for tokem will be huge
        yield token
def tokenize2(text):    
    cleaned_str=re.sub('[^a-z\s]+',' ',test,flags=re.IGNORECASE) #every char except alphabets is replaced
    cleaned_str=re.sub('(\s+)',' ',cleaned_str) #multiple spaces are replaced by single space
    cleaned_str=cleaned_str.lower() #converting the cleaned string to lower case    
    return cleaned_str

In [38]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rmallik\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [43]:
def train(dataset,labels): 
    if not isinstance(dataset,np.ndarray): dataset=np.array(dataset)
    if not isinstance(labels,np.ndarray): labels=np.array(labels)
    class_list= np.unique(labels)
    bow_dicts=np.array([defaultdict(lambda:0) for index in range(len(class_list))]) 
    word_count_class=[]
    prob_class=[]
    total_voc=[]
    for i,class_i in enumerate(class_list):
        data_class=dataset[labels==class_i]
        data_class = [list(tokenize(doc)) for doc in data_class] 
        word_class=[word for example in data_class for word in example]
        #total word count in class
        word_count_class.append(len(word_class))
        # BOW of a class: (word:count_word)
        for word in word_class:
            bow_dicts[i][word]+=1 
        # for vocabulraty count 
        total_voc+=word_class        
        #prob of class 
        prob_class.append(len(labels[labels==class_i])/len(labels))
    dataset_class=[]  
    for i,class_i in enumerate(class_list):        
        dataset_class.append([bow_dicts[i],prob_class[i],word_count_class[i]+len(list(set(total_voc)))+1])
    return dataset_class

In [48]:
def likelihood_prob(train_data,example,classes):
    likelihood_class=np.zeros(len(classes))
    for i,class_i in enumerate(classes):
        for token in example:
            token_prob=np.log((train_data[i][0].get(token,0) + 1)/ train_data[i][2])
            likelihood_class[i]+=token_prob
    return likelihood_class

In [44]:
def test(train_data,dataset,labels):
    class_list=np.unique(labels)
    #clean
    dataset= [list(tokenize(examples)) for examples in dataset]
    
    prediction=[]
    predict_class=np.zeros(len(class_list))
    for data in dataset:
        #predict the
        posterior_prob=np.zeros(len(class_list))
        lkl_prob=likelihood_prob(train_data,data,class_list) 
        for i , val in enumerate(lkl_prob):
            posterior_prob[i]=lkl_prob[i]+ np.log(train_data[i][1])        
        predict_class_arg=np.argmax(posterior_prob)
        prediction.append(class_list[predict_class_arg])
    return prediction

In [ ]:
#-----prepare data and train and test and find accuarcy as shown
#train_data_result=train(dataset,labels)
#prediction=test(train_data_result,dataset,labels)
#accuracy = accuracy_score(prediction,list(labels_test))
#accuracy

# Training on fetch_20newsgroups data 
note:fetch_20newsgroupsis a dataset that has 20 categories but we will restrict the categories
to 4 for the time being 

In [1]:
from sklearn.datasets import fetch_20newsgroups
categories=['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med'] 
newsgroups_train=fetch_20newsgroups(subset='train',categories=categories)


In [12]:
train_data=newsgroups_train.data #getting all trainign examples
train_labels=newsgroups_train.target #getting training labels
print ("Total Number of Training Examples: ",len(train_data)) # Outputs -> Total Number of Training Examples:  2257
print ("Total Number of Training Labels: ",len(train_labels)) # Outputs -> #Total Number of Training Labels:  2257
type(train_labels)

Total Number of Training Examples:  2257
Total Number of Training Labels:  2257


numpy.ndarray

In [45]:
train_data_info=train(train_data,train_labels)

In [46]:
newsgroups_test=fetch_20newsgroups(subset='test',categories=categories) #loading test data
test_data=newsgroups_test.data #get test set examples
test_labels=newsgroups_test.target #get test set labels

print ("Number of Test Examples: ",len(test_data)) # Output : Number of Test Examples:  1502
print ("Number of Test Labels: ",len(test_labels)) # Output : Number of Test Labels:  1502
prediction=test(train_data_info,test_data,test_labels)

Number of Test Examples:  1502
Number of Test Labels:  1502


In [47]:
accuracy = accuracy_score(prediction,list(test_labels))
accuracy

0.9254327563249002

In [52]:
test_data[0],test_labels[0],prediction[0]

("From: brian@ucsd.edu (Brian Kantor)\nSubject: Re: HELP for Kidney Stones ..............\nOrganization: The Avant-Garde of the Now, Ltd.\nLines: 12\nNNTP-Posting-Host: ucsd.edu\n\nAs I recall from my bout with kidney stones, there isn't any\nmedication that can do anything about them except relieve the pain.\n\nEither they pass, or they have to be broken up with sound, or they have\nto be extracted surgically.\n\nWhen I was in, the X-ray tech happened to mention that she'd had kidney\nstones and children, and the childbirth hurt less.\n\nDemerol worked, although I nearly got arrested on my way home when I barfed\nall over the police car parked just outside the ER.\n\t- Brian\n",
 2,
 2)